# Grok銘柄選定バックテスト分析（Plotly版）

## 2025-10-23予想 → 2025-10-24実績（5分足含む）

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
from datetime import datetime, time
import warnings
warnings.filterwarnings('ignore')

import plotly
print("Plotly version:", plotly.__version__)

## 1. Grok予想銘柄（英語表記版）

In [ ]:
# Grok予想データ（英語表記）
grok_predictions = [
    {"ticker": "9348", "name_jp": "ispace", "name_en": "ispace", "mentioned_by": ["@kabuchenko"], "category": "Premium+IR+X+Policy", "policy_link": "High", "sentiment_score": 0.85},
    {"ticker": "3929", "name_jp": "Synspective", "name_en": "Synspective", "mentioned_by": ["@kaikai2120621"], "category": "Premium+Bio+Geo", "policy_link": "High", "sentiment_score": 0.82},
    {"ticker": "5595", "name_jp": "QPS研究所", "name_en": "QPS Kenkyujo", "mentioned_by": [], "category": "Theme+X+Policy", "policy_link": "Med", "sentiment_score": 0.70},
    {"ticker": "6237", "name_jp": "ウエスト", "name_en": "West", "mentioned_by": ["@kabu777b"], "category": "Premium+IR+X+Policy", "policy_link": "High", "sentiment_score": 0.88},
    {"ticker": "6264", "name_jp": "イワキ", "name_en": "Iwaki", "mentioned_by": [], "category": "IR+X+Policy", "policy_link": "Med", "sentiment_score": 0.78},
    {"ticker": "186A", "name_jp": "アストロスケールHD", "name_en": "Astroscale HD", "mentioned_by": ["@daykabu2021"], "category": "Premium+Theme+X", "policy_link": "High", "sentiment_score": 0.75},
    {"ticker": "2459", "name_jp": "アウンコンサルティング", "name_en": "Aun Consulting", "mentioned_by": ["@jestryoR"], "category": "Premium+IR+X+Policy", "policy_link": "Med", "sentiment_score": 0.80},
    {"ticker": "3079", "name_jp": "ディーブイエックス", "name_en": "DVx", "mentioned_by": [], "category": "IR+X+Geo", "policy_link": "Low", "sentiment_score": 0.72},
    {"ticker": "3664", "name_jp": "モブキャストHD", "name_en": "Mobcast HD", "mentioned_by": [], "category": "News+X+Policy", "policy_link": "Med", "sentiment_score": 0.70},
    {"ticker": "2158", "name_jp": "FRONTEO", "name_en": "FRONTEO", "mentioned_by": ["@tesuta001"], "category": "Premium+News+X", "policy_link": "Low", "sentiment_score": 0.76},
    {"ticker": "3769", "name_jp": "ランディックス", "name_en": "RANDIX", "mentioned_by": [], "category": "Earnings+X+Policy", "policy_link": "Med", "sentiment_score": 0.74},
    {"ticker": "4398", "name_jp": "情報戦略テクノロジー", "name_en": "IT Strategy", "mentioned_by": [], "category": "News+X+Policy", "policy_link": "Low", "sentiment_score": 0.68}
]

df_predictions = pd.DataFrame(grok_predictions)
df_predictions['has_mention'] = df_predictions['mentioned_by'].apply(lambda x: len(x) > 0)
df_predictions['mentioned_by_str'] = df_predictions['mentioned_by'].apply(lambda x: ', '.join(x) if x else 'None')

print("\n【Grok Prediction List】")
display(df_predictions[['ticker', 'name_en', 'name_jp', 'mentioned_by_str', 'policy_link', 'sentiment_score', 'category']])

## 2. 日次データ取得（2025-10-24）

In [ ]:
# 2025-10-24の日次データを取得
target_date = "2025-10-24"
results = []

for idx, row in df_predictions.iterrows():
    ticker_code = row['ticker']
    ticker_symbol = f"{ticker_code}.T"
    
    try:
        ticker = yf.Ticker(ticker_symbol)
        hist = ticker.history(start=target_date, end="2025-10-25")
        
        if not hist.empty:
            data = hist.iloc[0]
            open_price = data['Open']
            close_price = data['Close']
            high_price = data['High']
            low_price = data['Low']
            volume = int(data['Volume'])
            
            change_pct = ((close_price - open_price) / open_price) * 100
            range_pct = ((high_price - low_price) / open_price) * 100
            
            results.append({
                'ticker': ticker_code,
                'name_en': row['name_en'],
                'name_jp': row['name_jp'],
                'open': open_price,
                'high': high_price,
                'low': low_price,
                'close': close_price,
                'volume': volume,
                'change_pct': change_pct,
                'range_pct': range_pct,
                'result': 'Up' if change_pct > 0 else 'Down',
                'mentioned_by_str': row['mentioned_by_str'],
                'policy_link': row['policy_link'],
                'sentiment_score': row['sentiment_score'],
                'category': row['category']
            })
        else:
            results.append({
                'ticker': ticker_code,
                'name_en': row['name_en'],
                'name_jp': row['name_jp'],
                'open': None, 'high': None, 'low': None, 'close': None, 'volume': None,
                'change_pct': None, 'range_pct': None, 'result': 'No Data',
                'mentioned_by_str': row['mentioned_by_str'],
                'policy_link': row['policy_link'],
                'sentiment_score': row['sentiment_score'],
                'category': row['category']
            })
    except Exception as e:
        print(f"Error: {ticker_code} - {e}")
        results.append({
            'ticker': ticker_code,
            'name_en': row['name_en'],
            'name_jp': row['name_jp'],
            'open': None, 'high': None, 'low': None, 'close': None, 'volume': None,
            'change_pct': None, 'range_pct': None, 'result': 'Error',
            'mentioned_by_str': row['mentioned_by_str'],
            'policy_link': row['policy_link'],
            'sentiment_score': row['sentiment_score'],
            'category': row['category']
        })

df_results = pd.DataFrame(results)

print("\n【Daily Data: 2025-10-24】")
display(df_results)

## 3. 5分足データ取得（前場 9:00-11:30）

In [ ]:
# 5分足データ取得（2025-10-24の前場）
intraday_data = {}

for idx, row in df_predictions.iterrows():
    ticker_code = row['ticker']
    ticker_symbol = f"{ticker_code}.T"
    
    try:
        ticker = yf.Ticker(ticker_symbol)
        # 5分足データを取得（過去5日分）
        hist_5m = ticker.history(period="5d", interval="5m")
        
        if not hist_5m.empty:
            # 2025-10-24のデータのみフィルタ
            hist_5m.index = hist_5m.index.tz_localize(None)  # タイムゾーンを削除
            target_day_data = hist_5m[hist_5m.index.date == pd.Timestamp(target_date).date()]
            
            # 前場（9:00-11:30）のみフィルタ
            morning_session = target_day_data[
                (target_day_data.index.time >= time(9, 0)) & 
                (target_day_data.index.time <= time(11, 30))
            ]
            
            if not morning_session.empty:
                intraday_data[ticker_code] = {
                    'data': morning_session,
                    'name_en': row['name_en'],
                    'name_jp': row['name_jp']
                }
                print(f"{ticker_code} ({row['name_en']}): {len(morning_session)} 5-min bars")
    except Exception as e:
        print(f"Error fetching 5m data for {ticker_code}: {e}")

print(f"\nTotal stocks with 5m data: {len(intraday_data)}")

## 4. パフォーマンスサマリー

In [ ]:
df_valid = df_results[df_results['change_pct'].notna()].copy()

total_stocks = len(df_valid)
win_count = (df_valid['change_pct'] > 0).sum()
lose_count = (df_valid['change_pct'] <= 0).sum()
win_rate = (win_count / total_stocks * 100) if total_stocks > 0 else 0
avg_change = df_valid['change_pct'].mean()
avg_range = df_valid['range_pct'].mean()
volatile_count = (df_valid['range_pct'] >= 2.0).sum()
volatile_rate = (volatile_count / total_stocks * 100) if total_stocks > 0 else 0

df_valid['has_mention'] = df_valid['mentioned_by_str'] != 'None'
mentioned_stocks = df_valid[df_valid['has_mention']]
not_mentioned_stocks = df_valid[~df_valid['has_mention']]

mentioned_win_rate = ((mentioned_stocks['change_pct'] > 0).sum() / len(mentioned_stocks) * 100) if len(mentioned_stocks) > 0 else 0
not_mentioned_win_rate = ((not_mentioned_stocks['change_pct'] > 0).sum() / len(not_mentioned_stocks) * 100) if len(not_mentioned_stocks) > 0 else 0

print("\n" + "="*60)
print("【Performance Summary】")
print("="*60)
print(f"Total Stocks: {total_stocks}")
print(f"Wins: {win_count} stocks ({win_rate:.1f}%)")
print(f"Losses: {lose_count} stocks ({100-win_rate:.1f}%)")
print(f"Avg Change: {avg_change:+.2f}%")
print(f"Avg Range: {avg_range:.2f}%")
print(f"Volatile (Range>=2%): {volatile_count} stocks ({volatile_rate:.1f}%)")
print("\n【Premium User Mention】")
print(f"With Mention Win Rate: {mentioned_win_rate:.1f}% ({len(mentioned_stocks)} stocks)")
print(f"Without Mention Win Rate: {not_mentioned_win_rate:.1f}% ({len(not_mentioned_stocks)} stocks)")
print("="*60)

## 5. Plotly可視化：変化率ランキング（日本語+英語）

In [ ]:
df_sorted = df_valid.sort_values('change_pct', ascending=True)

# 日本語名と英語名を併記
df_sorted['display_name'] = df_sorted['name_en'] + ' (' + df_sorted['name_jp'] + ')'

colors = ['green' if x > 0 else 'red' for x in df_sorted['change_pct']]

fig = go.Figure()

fig.add_trace(go.Bar(
    y=df_sorted['display_name'],
    x=df_sorted['change_pct'],
    orientation='h',
    marker=dict(color=colors),
    text=df_sorted['change_pct'].apply(lambda x: f"{x:+.2f}%"),
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>Change: %{x:.2f}%<extra></extra>'
))

fig.update_layout(
    title='Grok Prediction Results: 2025-10-23 -> 2025-10-24',
    xaxis_title='Change % (Close - Open)',
    yaxis_title='Stock',
    height=600,
    showlegend=False,
    template='plotly_white'
)

fig.add_vline(x=0, line_dash="dash", line_color="black", line_width=1)

fig.show()

## 6. センチメントスコア vs 実績

In [ ]:
fig = px.scatter(
    df_valid,
    x='sentiment_score',
    y='change_pct',
    color='policy_link',
    size='range_pct',
    hover_data=['name_en', 'name_jp', 'mentioned_by_str'],
    text='name_en',
    color_discrete_map={'High': 'red', 'Med': 'orange', 'Low': 'blue'},
    title='Sentiment Score vs Actual Performance',
    labels={
        'sentiment_score': 'Sentiment Score (Grok)',
        'change_pct': 'Actual Change % (2025-10-24)',
        'policy_link': 'Policy Link'
    }
)

fig.update_traces(textposition='top center', textfont_size=9)
fig.add_hline(y=0, line_dash="dash", line_color="black", line_width=1)
fig.update_layout(height=600, template='plotly_white')

fig.show()

## 7. 5分足チャート（前場の流れ）

In [ ]:
# 上位3銘柄と下位3銘柄の5分足チャートを表示
df_sorted_by_change = df_valid.sort_values('change_pct', ascending=False)
top_3 = df_sorted_by_change.head(3)['ticker'].tolist()
bottom_3 = df_sorted_by_change.tail(3)['ticker'].tolist()
selected_tickers = top_3 + bottom_3

for ticker_code in selected_tickers:
    if ticker_code in intraday_data:
        data_5m = intraday_data[ticker_code]['data']
        name_en = intraday_data[ticker_code]['name_en']
        name_jp = intraday_data[ticker_code]['name_jp']
        
        # 日次データから変化率を取得
        daily_change = df_results[df_results['ticker'] == ticker_code]['change_pct'].values[0]
        
        fig = go.Figure()
        
        # ローソク足チャート
        fig.add_trace(go.Candlestick(
            x=data_5m.index,
            open=data_5m['Open'],
            high=data_5m['High'],
            low=data_5m['Low'],
            close=data_5m['Close'],
            name='Price'
        ))
        
        fig.update_layout(
            title=f'{ticker_code} - {name_en} ({name_jp}) | Daily Change: {daily_change:+.2f}%',
            xaxis_title='Time (Morning Session)',
            yaxis_title='Price (JPY)',
            height=400,
            template='plotly_white',
            xaxis_rangeslider_visible=False
        )
        
        fig.show()
    else:
        print(f"No 5m data for {ticker_code}")

## 8. インタラクティブテーブル

In [ ]:
# Plotlyのテーブル表示
table_df = df_results[['ticker', 'name_en', 'name_jp', 'mentioned_by_str', 'policy_link', 
                        'sentiment_score', 'open', 'close', 'change_pct', 'range_pct', 'result']].copy()

# セルの色を設定
def get_color(val, column):
    if column == 'result':
        if val == 'Up':
            return 'lightgreen'
        elif val == 'Down':
            return 'lightcoral'
        else:
            return 'lightgray'
    return 'white'

fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(table_df.columns),
        fill_color='paleturquoise',
        align='left',
        font=dict(size=12, color='black')
    ),
    cells=dict(
        values=[table_df[col] for col in table_df.columns],
        fill_color='white',
        align='left',
        font=dict(size=11)
    )
)])

fig.update_layout(
    title='Detailed Results Table',
    height=500
)

fig.show()

## 9. 結論

- **勝率**: プレミアムユーザー言及の有無による差を確認
- **センチメントスコア**: 高スコアが実際の上昇と相関しているか
- **政策連動度**: Highの銘柄が実際に動いたか
- **5分足**: 前場の値動きパターンからスキャルピングタイミングを分析

次回のプロンプト改善に活用してください！